In [0]:
import numpy as np
import os
import sys
import plotly.express as px

package_path = os.path.join(os.path.dirname(os.getcwd()))
sys.path.append(package_path)

from src.models.markovitz_bullet import MarkovitzSimulator
from src.preprocess_data import preprocess
from src.utils.data_utils import *
from src.financials.ratio_metrics import RatioMetrics
from src.preprocess_data.preprocess import fund_scaler

sys.path.append("/Workspace/Repos/Libraries")

from Toolbox.src.viz.charter import ardian_charter_graph, ardian_charter_plotly, CORAL, DARK_GREY, STONE, BLUE_GREY, BROWN, PURPLE, FULL_PALETTE
FULL_PALETTE = np.array(FULL_PALETTE)

%load_ext autoreload
%autoreload 2

In [0]:
%run "/Users/rigaudg@ardian.com/Portfolio value at risk/lake_config"   

In [0]:
%run "/Repos/rigaudg@ardian.com/PortfolioValueAtRisk/PortfolioValueAtRisk/ddp_version/00-Workflow config"

In [0]:
file_returns_table = f"abfss://portfolio-var@{account}.dfs.core.windows.net/output/dashboard/metrics_DDP_database.parquet"
metrics_table = read_lake_data(spark, file_returns_table)

file_returns_table = f"abfss://portfolio-var@{account}.dfs.core.windows.net/output/dashboard/value_at_risk_DDP_database.parquet"
returns_table = read_lake_data(spark, file_returns_table)

macro_file = f"abfss://portfolio-var@{account}.dfs.core.windows.net/processed/{LAST_REPORTING_EXTRACT}/macro_factors.parquet"
macro_dataframe = read_lake_data(spark, macro_file).set_index("date")

In [0]:
pred = preprocess.PreprocessData() 
fund_table = pred.preprocess_delta(metrics_table, returns_table)
pe_public_dict, quarterly_public_returns = pred.fund_pe_combiner(fund_table, macro_dataframe) #Return a dictionnary of dataframes (1 per Ardian fund) + a dataframe of public indexes

In [0]:
return_df = metrics_table.loc[lambda x : x.level == 'asset', :"return"].drop(columns = ["level", "portfolioName"]).drop_duplicates(subset = ["quarterDate", "entity"])
return_df = pd.pivot(return_df, columns = "entity", index =  "quarterDate")

assets = [(m, s, "asset") for m, s in zip(np.mean(return_df, axis = 0).values, np.std(return_df, axis = 0).values)]
funds = [(np.mean(pe_public_dict.get(k)["return"].values / 100), np.std(pe_public_dict.get(k)["return"].values / 100), "fund") for k in pe_public_dict.keys()]
full_risk_list = assets + funds

full_risk_df = pd.DataFrame(full_risk_list, columns = ["mean", "std", "ticker"])
full_risk_df["entity"] = return_df.columns.get_level_values(1).to_list() + list(pe_public_dict.keys())

fig  = px.scatter(full_risk_df, "std", "mean", color = "ticker", hover_name="entity")
ardian_charter_plotly(fig)


In [0]:
fund_df = pd.concat(pe_public_dict).reset_index().iloc[:, 1:4]
fund_df = pd.pivot(fund_df, "quarterDate", "entity", "return").iloc[:,:-1]
fund_df.head() 

asset_df = metrics_table.loc[lambda x : x.level == "asset"].drop_duplicates(["quarterDate", "entity"])
asset_df = pd.pivot(asset_df, "quarterDate", "entity", "return")
asset_arr = asset_df.values

fig = px.scatter(y = np.nanmean(asset_arr, axis = 0),  x = np.nanstd(asset_arr, axis = 0), color = 2 * np.nanmean(asset_arr, axis = 0) / np.nanstd(asset_arr, axis = 0))
fig

In [0]:
MarkovitzSimulator(asset_arr, periods = 4).make_random_portfolios(1000).compute_sharpe_ratios().plot()

In [0]:
res = [MarkovitzSimulator(asset_arr[:,i], periods = 4).make_random_portfolios(1).compute_sharpe_ratios() for i in range(asset_arr.shape[1])]
simulations = [(val.simulations[0][0], val.simulations[1][0]) for val in res]
sharpe_ratios = [val.sharpe_ratio[0] for val in res]
simulations = pd.DataFrame(simulations, columns= ["mu", "sigma"]).assign(sharpe_ratio = sharpe_ratios, 
                                                                        entity = asset_df.columns.to_list(),
                                                                        ticker = "asset")

funds = [(np.mean(pe_public_dict.get(k)["return"].values / 100), np.std(pe_public_dict.get(k)["return"].values / 100), k) for k in pe_public_dict.keys()]
funds = pd.DataFrame(funds, columns = ["mu", "sigma", "entity"]).assign(sharpe_ratio = lambda x : x["mu"] / x['sigma'], ticker = "fund")

simulations = pd.concat([simulations, funds])
simulations.iloc[:, :2] = simulations.iloc[:, :2] * 100

fig  = px.scatter(simulations, "sigma", "mu", title="Markovitz plot", hover_name="entity", color = "ticker", symbol = "ticker")
ardian_charter_plotly(fig)